<a href="https://colab.research.google.com/github/almc6742/cu-git-workshop/blob/master/Exercise_4_Geographic_Selections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import json
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
msas = 'drive/My Drive/Colab Notebooks/msas.csv'
df = pd.read_csv(msas)

Organic Parameters:
1. 50K to 74.9K
2. 18-34
3. Hispanic


In [0]:
tableids = ['B01001', 'B03002', 'B19001']
tableidstring = ','.join(tableids)

URL request for desired data tables from CensusReporter API:

In [0]:
requesturl = 'https://api.censusreporter.org/1.0/data/show/latest?table_ids=%s&geo_ids=310|01000US' % tableidstring

In [0]:
loadedjson = requests.get(requesturl)

Store json from API request and parse:

In [0]:
parsedjson = loadedjson.json()

In [7]:
#Question 1
#sort CBSA regions based on their 2017 population estimate
df.sort_values(by=['POPESTIMATE2017'], ascending=False).head()

,CBSA,NAME,POPESTIMATE2017
249,35620,"New York-Newark-Jersey City, NY-NJ-PA",20320876
210,31080,"Los Angeles-Long Beach-Anaheim, CA",13353907
68,16980,"Chicago-Naperville-Elgin, IL-IN-WI",9533040
86,19100,"Dallas-Fort Worth-Arlington, TX",7399662
159,26420,"Houston-The Woodlands-Sugar Land, TX",6892427


In [8]:
income50 = pd.DataFrame({ 'income50': { id_: data['B19001']['estimate']['B19001011'] for id_, data in parsedjson['data'].items() }})
income50 = income50.reset_index()
income60 = pd.DataFrame({ 'income60': { id_: data['B19001']['estimate']['B19001012'] for id_, data in parsedjson['data'].items() }})
income60 = income60.reset_index()
incomes = income60.merge(income50, how = 'inner', on = ['index'])
incomes = incomes.rename(columns={'index':'CBSA'})
incomes['CBSA'] = incomes['CBSA'].str.replace(r'31000US', '').astype(int)
incomes['Total'] = incomes['income50'] + incomes['income60']
incomes['Total'].max()

1050888.0

In [9]:
incomes[incomes['Total']==1050888.0]

,CBSA,income60,income50,Total
616,35620,606435.0,444453.0,1050888.0


In [10]:
#make a call from parsed json for data related to population by salary for first parameter
parsedjson['data']['31000US35620']['B19001']['estimate']
#'B19001011' = $50,000 to $59,999 and 'B19001012' = $60,000 to $74,999

{'B19001001': 7168027.0,
 'B19001002': 478614.0,
 'B19001003': 330184.0,
 'B19001004': 296675.0,
 'B19001005': 291324.0,
 'B19001006': 267636.0,
 'B19001007': 266550.0,
 'B19001008': 251442.0,
 'B19001009': 248742.0,
 'B19001010': 216645.0,
 'B19001011': 444453.0,
 'B19001012': 606435.0,
 'B19001013': 824870.0,
 'B19001014': 670890.0,
 'B19001015': 481361.0,
 'B19001016': 630168.0,
 'B19001017': 862038.0}

1. The New York-Newark-Jersey City, NY-NJ-PA Metro Area has the most residents (1,050,888) with an income between $50,000 to $74,999.

In [11]:
#Question 2
#make a call from parsed json for data related to population by age
parsedjson['data']['31000US35620']['B01001']['estimate']
#focus on B01001007, B01001008, B01001009, B01001010, B01001011B01001012

{'B01001001': 20192042.0,
 'B01001002': 9762858.0,
 'B01001003': 632584.0,
 'B01001004': 611017.0,
 'B01001005': 624418.0,
 'B01001006': 385399.0,
 'B01001007': 248434.0,
 'B01001008': 133732.0,
 'B01001009': 132580.0,
 'B01001010': 411899.0,
 'B01001011': 750663.0,
 'B01001012': 714983.0,
 'B01001013': 660955.0,
 'B01001014': 643736.0,
 'B01001015': 677473.0,
 'B01001016': 699632.0,
 'B01001017': 642501.0,
 'B01001018': 237891.0,
 'B01001019': 314944.0,
 'B01001020': 189012.0,
 'B01001021': 242555.0,
 'B01001022': 303854.0,
 'B01001023': 213991.0,
 'B01001024': 149206.0,
 'B01001025': 141399.0,
 'B01001026': 10429184.0,
 'B01001027': 604184.0,
 'B01001028': 587836.0,
 'B01001029': 595623.0,
 'B01001030': 368965.0,
 'B01001031': 237913.0,
 'B01001032': 129681.0,
 'B01001033': 129093.0,
 'B01001034': 416225.0,
 'B01001035': 762402.0,
 'B01001036': 730200.0,
 'B01001037': 684982.0,
 'B01001038': 674162.0,
 'B01001039': 720797.0,
 'B01001040': 749155.0,
 'B01001041': 704947.0,
 'B01001042

In [12]:
age7 = pd.DataFrame({ '18to19': { id_: data['B01001']['estimate']['B01001007'] for id_, data in parsedjson['data'].items() }})
age7 = age7.reset_index()
age8 = pd.DataFrame({ '20': { id_: data['B01001']['estimate']['B01001008'] for id_, data in parsedjson['data'].items() }})
age8 = age8.reset_index()
age9 = pd.DataFrame({ '21': { id_: data['B01001']['estimate']['B01001009'] for id_, data in parsedjson['data'].items() }})
age9 = age9.reset_index()
age10 = pd.DataFrame({ '22to24': { id_: data['B01001']['estimate']['B01001010'] for id_, data in parsedjson['data'].items() }})
age10 = age10.reset_index()
age11 = pd.DataFrame({ '25to29': { id_: data['B01001']['estimate']['B01001011'] for id_, data in parsedjson['data'].items() }})
age11 = age11.reset_index()
age12 = pd.DataFrame({ '30to34': { id_: data['B01001']['estimate']['B01001012'] for id_, data in parsedjson['data'].items() }})
age12 = age12.reset_index()
ages1 = age7.merge(age8, how = 'inner', on = ['index'])
ages2 = ages1.merge(age9, how = 'inner', on = ['index'])
ages3 = ages2.merge(age10, how = 'inner', on = ['index'])
ages4 = ages3.merge(age11, how = 'inner', on = ['index'])
ages = ages4.merge(age12, how = 'inner', on = ['index'])
ages = ages.rename(columns={'index':'CBSA'})
ages['CBSA'] = ages['CBSA'].str.replace(r'31000US', '').astype(int)
ages['Total'] = ages['18to19'] + ages['20'] + ages['21'] + ages['22to24'] + ages['25to29'] + ages['30to34']
ages['Total'].max()

2392291.0

In [13]:
ages[ages['Total']==2392291.0]

,CBSA,18to19,20,21,22to24,25to29,30to34,Total
616,35620,248434.0,133732.0,132580.0,411899.0,750663.0,714983.0,2392291.0


2. The New York-Newark-Jersey City, NY-NJ-PA Metro Area has the most residents (2,392,291) between the ages of 18-34.

In [14]:
#Question 3
#make a call from parsedjson for data related to population by ethnicity
parsedjson['data']['31000US31080']['B03002']['estimate']
#focus on 12

{'B03002001': 13261538.0,
 'B03002002': 7288787.0,
 'B03002003': 3983380.0,
 'B03002004': 849139.0,
 'B03002005': 26499.0,
 'B03002006': 2058236.0,
 'B03002007': 33664.0,
 'B03002008': 34233.0,
 'B03002009': 303636.0,
 'B03002010': 17353.0,
 'B03002011': 286283.0,
 'B03002012': 5972751.0,
 'B03002013': 3209939.0,
 'B03002014': 33104.0,
 'B03002015': 56180.0,
 'B03002016': 23450.0,
 'B03002017': 3647.0,
 'B03002018': 2440456.0,
 'B03002019': 205975.0,
 'B03002020': 114543.0,
 'B03002021': 91432.0}

In [15]:
hispanic = pd.DataFrame({ 'Total': { id_: data['B03002']['estimate']['B03002012'] for id_, data in parsedjson['data'].items() }})
hispanic = hispanic.reset_index()
hispanic = hispanic.rename(columns={'index':'CBSA'})
hispanic['CBSA'] = hispanic['CBSA'].str.replace(r'31000US', '').astype(int)
hispanic['Total'].max()

5972751.0

In [16]:
hispanic[hispanic['Total']==5972751.0]

,CBSA,Total
506,31080,5972751.0


3. The Los Angeles-Long Beach-Anaheim, CA Metro Area has the greatest Hispanic population at 5,972,751 people.

In [17]:
#Question 4
##Parameter 1 Calculation
incomes['Income Mean Percentage'] = (incomes['Total']/incomes['Total'].max())*100

#Parameter 2 Calculation
ages['Age Mean Percentage'] = (ages['Total']/ages['Total'].max())*100

#Parameter 3 Calculation
hispanic['Hispanic Mean Percentage'] = (hispanic['Total']/hispanic['Total'].max())*100

Q4 = pd.DataFrame([incomes['CBSA'], incomes['Income Mean Percentage'],ages['Age Mean Percentage'],hispanic['Hispanic Mean Percentage']]).transpose()
Q4['Mean of All 3'] = (Q4['Hispanic Mean Percentage']+Q4['Income Mean Percentage']+Q4['Age Mean Percentage'])/3
Q4['Mean of All 3'].max()

93.8099489944695

In [18]:
Q4[Q4['Mean of All 3']==93.8099489944695]

,CBSA,Income Mean Percentage,Age Mean Percentage,Hispanic Mean Percentage,Mean of All 3
616,35620.0,100.0,100.0,81.429847,93.809949


4. The New York-Newark-Jersey City, NY-NJ-PA Metro Area has the highest mean percentage across all three categories at 93.81%.